In [1]:
%load_ext autoreload
%autoreload 2

import seaborn as sns
import metapack as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 

import re
import xml.etree.ElementTree as ET
import pandas as pd

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()


In [2]:
#pkg = mp.jupyter.open_package()
pkg = mp.jupyter.open_source_package()
pkg.set_sys_path()
from pylib import *
pkg

`isr.umich.edu-psid-inequality-1.1.1` Last Update: 2023-06-06T20:57:36

__



## Updating Data

To update the data, use the data extration tool at the (PSID website. )[https://simba.isr.umich.edu/Cart/co.aspx], adding any new variables to your data cart. The (variable entry text box)[https://simba.isr.umich.edu/DC/l.aspx] is a handy way to copy the existing varibles into a new cart.  When you checkout your data cart, ( click on the cart icon ) fetch your dataset with these options: 

* Codebook Type: XML
* Data output type: dBase Data File, create compressed file

Download the ZIP file ( you might have to recompress it if your browser unpacks it ) and move it into the ``data`` directory. Set the reference to the new ZIP file, then re-build the package.
## Documentation Links

* [PSID Homepage](https://psidonline.isr.umich.edu/default.aspx) 

 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)

## Resources

* **[psid_ineq](notebooks/extract.ipynb#df)**. PSID, Inequality extract
* **[_labels](notebooks/extract.ipynb#labels)**. Categorical value labels
* **[data_dictionary](notebooks/extract.ipynb#mdf)**. List of Variables

## References
<ul><li> <strong><a href="https://psidonline.isr.umich.edu/help/xyr/psid.xlsx">variables</a></strong>. Variable descriptions, in a cross-year format.</li><li> <strong>source</string>, <em>data/J321027.zip</em>. Downloaded source file</li><ul>

In [3]:
# create URLs to the files inside the data distribution ZIP file
surl   = pkg.reference('source').resolved_url
base_name = surl.resource_file.split('.')[0]
database_url = surl.set_target_file(base_name+'.dbf')

from simpledbf import Dbf5
dbf = Dbf5(database_url.get_target().fspath)
df = dbf.to_dataframe()

# Make everything as numerical as possible
for c in df.columns:
    df[c] = pd.to_numeric(df[c])

# Recommended construction of unique persion id
df['pid'] = df['ER30001'] + df['ER30002']


In [9]:
# opening_columns. These should always be at the start of the schema.
init_cols = ['ER30000','ER30001','ER30002']


In [6]:
mdf = extract_metadata(pkg)
labels = extract_labels(pkg)

In [11]:
# Re-order the data dictionary
mdf_sel = mdf[mdf.name.isin(df.columns)].sort_values(['category', 'label', 'year'])

is_init = mdf_sel['name'].isin(init_cols)
is_si = mdf_sel.category == 'SURVEY INFORMATION'

a = mdf_sel.loc[ is_init ]
b = mdf_sel.loc[~is_init &  is_si]
c = mdf_sel.loc[~is_init & ~is_si]

mdf_sel = pd.concat([a,b,c])

display(mdf_sel)

st = pkg.resource('psid_ineq').schema_term
df = update_schema(st, mdf_sel, df)

df.head()


,year,type_id,name,label,qtext,etext,type,category,text,head_wife,var_count,path
1,1968,0,ER30001,1968 INTERVIEW NUMBER,1968 Interview Number (1968 ID Number),This variable is the 1968 family ID number. T...,INDIVIDUAL,SURVEY INFORMATION,01>SURVEY INFORMATION:\n02>Interview Informati...,,41.0,/survey information:/interview information/id ...
2,1968,0,ER30002,PERSON NUMBER 68,Person Number,,INDIVIDUAL,SURVEY INFORMATION,01>SURVEY INFORMATION:\n02>Interview Informati...,,1.0,/survey information:/interview information/per...
0,1968,0,ER30000,RELEASE NUMBER,Release Number,For Release 2 we have updated Relation to Hea...,INDIVIDUAL,SURVEY INFORMATION,01>SURVEY INFORMATION:\n02>Interview Informati...,,1.0,/survey information:/interview information/rel...
56,2001,2,ER33601,2001 INTERVIEW NUMBER,2001 Interview Number (2001 ID Number),The values for this variable represent the 200...,INDIVIDUAL,SURVEY INFORMATION,01>SURVEY INFORMATION:\n02>Interview Informati...,,41.0,/survey information:/interview information/id ...
119,2003,2,ER33701,2003 INTERVIEW NUMBER,2003 Interview Number (2003 ID Number),The values for this variable represent the 200...,INDIVIDUAL,SURVEY INFORMATION,01>SURVEY INFORMATION:\n02>Interview Informati...,,41.0,/survey information:/interview information/id ...
...,...,...,...,...,...,...,...,...,...,...,...,...
530,2015,1,ER64873,L72 OCCUPATION 1ST FULL TIME JOB-HD,L72. Thinking of (your/his/her) first full-tim...,The 3-digit occupation code from 2000 CENSUS ...,FAMILY PUBLIC,WORK,01>WORK\n02>Occupation\n03>1st full-time job\n...,HEAD,7.0,"/work/occupation/st full-time job/-digit, cen..."
177,2005,1,ER27446,L72-72A OCCUPATION 1ST FULL TIME JOB-HD,L72. Thinking of your first full-time regular ...,See the note at ER27352. The 3-digit occupati...,FAMILY PUBLIC,WORK,01>WORK\n02>Occupation\n03>1st full-time job\n...,HEAD,7.0,"/work/occupation/st full-time job/-digit, cen..."
246,2007,1,ER40618,L72-72A OCCUPATION 1ST FULL TIME JOB-HD,L72. Thinking of your first full-time regular ...,See the note at ER40527. The 3-digit occupati...,FAMILY PUBLIC,WORK,01>WORK\n02>Occupation\n03>1st full-time job\n...,HEAD,7.0,"/work/occupation/st full-time job/-digit, cen..."
315,2009,1,ER46596,L72-72A OCCUPATION 1ST FULL TIME JOB-HD,L72. Thinking of your first full-time regular ...,See the note at ER46504. The 3-digit occupati...,FAMILY PUBLIC,WORK,01>WORK\n02>Occupation\n03>1st full-time job\n...,HEAD,7.0,"/work/occupation/st full-time job/-digit, cen..."


,pid,ER30001,ER30002,ER30000,ER33601,ER33701,ER33801,ER33901,ER34001,ER34101,...,ER46502,ER51863,ER57603,ER51957,ER57713,ER64873,ER27446,ER40618,ER46596,ER23478
0,7,4,3,3,96,1392,289,148,13,218,...,832.0,832.0,NaN,936.0,NaN,NaN,936.0,936.0,936.0,936.0
1,8,4,4,3,5987,6278,2356,5399,1827,8397,...,0.0,0.0,0.0,516.0,516.0,NaN,516.0,516.0,516.0,516.0
2,9,4,5,3,6872,7782,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,742.0
3,10,4,6,3,5599,4812,4097,720,1840,2541,...,0.0,0.0,0.0,476.0,476.0,476.0,476.0,476.0,476.0,476.0
4,11,4,7,3,7091,0,6880,0,1840,2541,...,0.0,0.0,0.0,476.0,476.0,476.0,832.0,NaN,476.0,NaN
